In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [2]:
dataframe = pd.read_csv("D:\minkwan\졸업작품\coordikitty-ML-DL\dataframe_tops.csv")
dataframe

,image_file,label_file,large_category,medium_category,small_category,핏,촉감,신축성,비침,두께,계절,major_style,minor_style
0,TOPS_LESS_LESS_image_1001_0.jpg,1001_label_TOPS_LESS_LESS.json,TOPS,LESS,LESS,NaN,부드러움,있음,없음,얇음,여름,스포티,NaN
1,TOPS_LESS_LESS_image_1001_1.jpg,1001_label_TOPS_LESS_LESS.json,TOPS,LESS,LESS,NaN,부드러움,있음,없음,얇음,여름,스포티,NaN
2,TOPS_LESS_LESS_image_1002_0.jpg,1002_label_TOPS_LESS_LESS.json,TOPS,LESS,LESS,레귤러,보통,보통,보통,보통,"봄, 여름",스포티,NaN
3,TOPS_LESS_LESS_image_100_1.jpg,100_label_TOPS_LESS_LESS.json,TOPS,LESS,LESS,"루즈, 오버 사이즈",보통,보통,보통,보통,"봄, 여름, 가을",스포티,NaN
4,TOPS_LESS_LESS_image_1010_3.jpg,1010_label_TOPS_LESS_LESS.json,TOPS,LESS,LESS,"슬림, 레귤러",약간 부드러움,있음,거의 없음,약간 얇음,여름,스포티,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10004,TOPS_SHORT_T_SHIRTS_image_997_1.jpg,997_label_TOPS_SHORT_T_SHIRTS.json,TOPS,SHORT,T_SHIRTS,오버 사이즈,부드러움,있음,없음,보통,"봄, 여름, 가을, 겨울",캐주얼,스트릿
10005,TOPS_SHORT_T_SHIRTS_image_998_6.jpg,998_label_TOPS_SHORT_T_SHIRTS.json,TOPS,SHORT,T_SHIRTS,오버 사이즈,보통,보통,거의 없음,보통,"봄, 여름, 가을",캐주얼,스트릿
10006,TOPS_SHORT_T_SHIRTS_image_999_4.jpg,999_label_TOPS_SHORT_T_SHIRTS.json,TOPS,SHORT,T_SHIRTS,레귤러,부드러움,있음,없음,보통,"봄, 여름, 가을, 겨울",캐주얼,스트릿
10007,TOPS_SHORT_T_SHIRTS_image_99_1.jpg,99_label_TOPS_SHORT_T_SHIRTS.json,TOPS,SHORT,T_SHIRTS,루즈,보통,보통,거의 없음,보통,"봄, 여름",캐주얼,스트릿


In [3]:
# 라벨 인코딩을 위한 사전 생성
label_mapping = {label: idx for idx, label in enumerate(dataframe['major_style'].unique())}
dataframe['label'] = dataframe['major_style'].map(label_mapping)  # 새로운 숫자형 라벨 컬럼 추가

In [4]:
# 라벨 매핑 출력
print("라벨 매핑:")
for label, idx in label_mapping.items():
    print(f"'{label}': {idx}")

라벨 매핑:
'스포티': 0
'캐주얼': 1
'미니멀': 2
'포멀': 3


In [5]:
# CustomDataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_dir, label_column, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.label_column = label_column
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # 이미지 파일 이름을 데이터프레임의 'image_file' 컬럼에서 가져옴
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx]['image_file'])
        
        # 이미지 열기
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx]['label']  # 숫자형 라벨 사용

        if self.transform:
            image = self.transform(image)

        # 라벨을 Tensor로 변환
        label = torch.tensor(label, dtype=torch.long)

        return image, label  # (image, label) 튜플 반환

In [6]:
image_dir = "D:\minkwan\졸업작품\coordikitty-ML-DL\image_root_dir"
label_column = 'major_style'  # 사용할 라벨 컬럼

In [7]:
# 이미지 전처리 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [8]:
# 데이터셋 및 데이터로더 초기화
dataset = CustomDataset(dataframe, image_dir, label_column, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
# ResNeXt 모델 정의
model = models.resnext50_32x4d(pretrained=True)  # ResNeXt50 모델
num_classes = len(dataframe[label_column].unique())
model.fc = nn.Linear(model.fc.in_features, num_classes)

c:\Users\User\anaconda3\envs\Pytorch_CK\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\envs\Pytorch_CK\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# 모델을 GPU 또는 CPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(device)

cuda


In [11]:
# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        images, labels = batch  # 배치를 언팩킹
        images, labels = images.to(device), labels.to(device)  # 이미지와 라벨을 device로 이동

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("학습 완료!")

Epoch [1/10], Loss: 0.2354
Epoch [2/10], Loss: 0.5199
Epoch [3/10], Loss: 0.3289
Epoch [4/10], Loss: 0.6136
Epoch [5/10], Loss: 0.0606
Epoch [6/10], Loss: 0.3402
Epoch [7/10], Loss: 0.2142
Epoch [8/10], Loss: 0.0880
Epoch [9/10], Loss: 0.2324
Epoch [10/10], Loss: 0.2043
학습 완료!


In [12]:
# 학습된 모델 가중치 저장
model_save_path = 'resnext_model_tops_major_style.pth'  # 저장할 파일 경로
torch.save(model.state_dict(), model_save_path)

print("학습 완료 및 모델 가중치 저장!")

학습 완료 및 모델 가중치 저장!
